<a href="https://colab.research.google.com/github/SoniaSun810/LangChain_Practice/blob/main/Copy_of_LangChain_SQL_Database_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! pip install langchain openai pymysql --upgrade -q

In [10]:
from secret_key import OpenAI_API_key
import os
os.environ["OPENAI_API_KEY"] = OpenAI_API_key

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
db_user = "admin"
db_password = "MSWE-ta-project!"
db_host = "aws-ta-matching.cpxfig1ppqqw.eu-north-1.rds.amazonaws.com"
db_name = "TA_Matching_System"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0.5)

In [ ]:
toolkit = SQLDatabaseToolkit(db = db, llm = llm)

In [ ]:
agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    verbose = True
)

**Test of LangChain SQL Agent - Basic Functions**

In [16]:
agent_executor.run("Describe the Table Courses")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Courses, Students_basic_info, Students_candidacy
Thought:I can query the "Courses" table to describe its structure.
Action: sql_db_schema
Action Input: "Courses"
Observation: 
CREATE TABLE `Courses` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	course_number VARCHAR(255), 
	course_name VARCHAR(255), 
	course_description TEXT, 
	prerequisites TEXT, 
	overlaps VARCHAR(255), 
	restrictions VARCHAR(255), 
	repeatability VARCHAR(255), 
	grading_options VARCHAR(255), 
	same_as VARCHAR(255), 
	roman_numerals VARCHAR(255), 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from Courses table:
id	course_number	course_name	course_description	prerequisites	overlaps	restrictions	repeatability	grading_options	same_as	roman_numerals
1	I&C SCI 9	Introduction to Computation for Scientists and Engineers	Introduces principles, techniques, and computational tools for quantitative ap

'The Courses table has the structure mentioned above and contains 3 sample rows.'

In [17]:
agent_executor.run("Which course would the student Smith1 be most suitable to be a Teaching Assistant for?")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Courses, Students_basic_info, Students_candidacy
Thought:I can query the Courses, Students_basic_info, and Students_candidacy tables to find the answer to the question.
Action: sql_db_schema
Action Input: "Courses, Students_basic_info, Students_candidacy"
Observation: 
CREATE TABLE `Courses` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	course_number VARCHAR(255), 
	course_name VARCHAR(255), 
	course_description TEXT, 
	prerequisites TEXT, 
	overlaps VARCHAR(255), 
	restrictions VARCHAR(255), 
	repeatability VARCHAR(255), 
	grading_options VARCHAR(255), 
	same_as VARCHAR(255), 
	roman_numerals VARCHAR(255), 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from Courses table:
id	course_number	course_name	course_description	prerequisites	overlaps	restrictions	repeatability	grading_options	same_as	roman_numerals
1	I&C SCI 9	Introduction to Computation for Scientist

'CS 295P Keystone Project with instructor A+ in the quarter 202303.'

In [18]:
agent_executor.run("Which course in the Courses Table would the student Smith1 be most suitable to be a Teaching Assistant for?")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Courses, Students_basic_info, Students_candidacy
Thought:I should query the schema for the Courses table to see what columns it has that might be relevant to the question.
Action: sql_db_schema
Action Input: "Courses"
Observation: 
CREATE TABLE `Courses` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	course_number VARCHAR(255), 
	course_name VARCHAR(255), 
	course_description TEXT, 
	prerequisites TEXT, 
	overlaps VARCHAR(255), 
	restrictions VARCHAR(255), 
	repeatability VARCHAR(255), 
	grading_options VARCHAR(255), 
	same_as VARCHAR(255), 
	roman_numerals VARCHAR(255), 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from Courses table:
id	course_number	course_name	course_description	prerequisites	overlaps	restrictions	repeatability	grading_options	same_as	roman_numerals
1	I&C SCI 9	Introduction to Computation for Scientists and Engineers	Introduces principles,

'There are no courses in the Courses Table that the student Smith1 would be suitable to be a Teaching Assistant for.'

**Tutorial, Langchain Beginner - Other Tools might be useful**

---


1. promptTemplate

In [20]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables = ['course_number'],
    template = "In Students Table, which student is a better teaching assistant candidate for {course_number}?"
)
prompt_template_name.format(course_number = "I&C SCI 45J")

'In Students Table, which student is a better teaching assistant candidate for I&C SCI 45J?'

2. LLMChain

In [24]:
from langchain.chains import LLMChain

student_name_chain = LLMChain(llm = llm, prompt = prompt_template_name)

student_name_chain.run("I&C SCI 45J")

'To determine the best teaching assistant candidate for I&C SCI 45J from the Students Table, we need more information about the criteria for a good teaching assistant candidate. The Students Table likely contains information such as student names, majors, GPA, previous teaching experience, and relevant coursework. Without this information, it is not possible to identify a specific student as the best candidate.'

In [22]:
from langchain.schema import prompt
agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    verbose = True,
    prompt = prompt_template_name
)

agent_executor.run("I&C SCI 45J")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Courses, Students_basic_info, Students_candidacy
Thought:I see three tables in the database: Courses, Students_basic_info, and Students_candidacy. I should query the schema of these tables to see if any of them contain information related to I&C SCI 45J.
Action: sql_db_schema
Action Input: "Courses, Students_basic_info, Students_candidacy"
Observation: 
CREATE TABLE `Courses` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	course_number VARCHAR(255), 
	course_name VARCHAR(255), 
	course_description TEXT, 
	prerequisites TEXT, 
	overlaps VARCHAR(255), 
	restrictions VARCHAR(255), 
	repeatability VARCHAR(255), 
	grading_options VARCHAR(255), 
	same_as VARCHAR(255), 
	roman_numerals VARCHAR(255), 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from Courses table:
id	course_number	course_name	course_description	prerequisites	overlaps	restrictions	repeatability	gradin

'The course "I&C SCI 45J" is called "Programming in Java as a Second Language". It is an introduction to the lexical, syntactic, semantic, and pragmatic characteristics of the Java language for experienced programmers.'

3. Sequential Chain

In [27]:
from langchain.chains import SimpleSequentialChain

prompt_template_student_background = PromptTemplate(
    input_variables = ['student_name'],
    template = "According to Students Table, summarize the backgound of {student_name}?"
)

student_background_chain = LLMChain(llm = llm, prompt = prompt_template_student_background)

chain = SimpleSequentialChain(chains = [student_name_chain, student_background_chain])
response = chain.run("I&C SCI 45J")
print(response)


The background of the Students Table is not provided in the given information. Without knowing the specific attributes and skills of each student in the table, it is not possible to determine a better teaching assistant candidate for I&C SCI 45J.


4. Sequential Chain

In [28]:
prompt_template_name = PromptTemplate(
    input_variables = ['course_number'],
    template = "In Students Table, which student is a better teaching assistant candidate for {course_number}?"
)

student_name_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key="student_name")

prompt_template_student_background = PromptTemplate(
    input_variables = ['student_name'],
    template = "According to Students Table, summarize the backgound of {student_name}?"
)

student_background_chain = LLMChain(llm = llm, prompt = prompt_template_student_background, output_key = "student_background")



In [ ]:
from langchain.chains import SequentialChain

SequentialChain(
    chains = [student_name_chain, student_background_chain],
    input_variables= ["course_number"],
    output_variables= ['student_name', 'student_background']
)

chain({'course_number' : "I&C SCI 45J"})


